# 使用Amazon Athena和AWS Glue Catalog查询S3中的数据集
我们将在Glue Catalog(Hive元数据存储)中创建一个Athena表,创建后可以在Athena可以查询S3中数据集文件的内容。我们将基于S3中的`Amazon Customer Reviews Dataset`在Glue Catalog中创建表。我们定义了与数据映射的列,指定数据的格式,并指定Amazon S3中文件所在的位置。

<img src="img/athena_register_parquet.png" width="60%" align="left">

<a name='1'></a>
## Set up Kernel and Required Dependencies

First, check that the correct kernel is chosen.

<img src="img/kernel_set_up.png" width="300"/>

You can click on that to see and check the details of the image, kernel, and instance type.

<img src="img/w3_kernel_and_instance_type.png" width="600"/>

In [2]:
import psutil

notebook_memory = psutil.virtual_memory()
print(notebook_memory)

if notebook_memory.total < 32 * 1000 * 1000 * 1000:
    print('*******************************************')    
    print('YOU ARE NOT USING THE CORRECT INSTANCE TYPE')
    print('PLEASE CHANGE INSTANCE TYPE TO  m5.2xlarge ')
    print('*******************************************')
else:
    correct_instance_type=True

svmem(total=33229983744, available=31371825152, percent=5.6, used=1443811328, free=19124060160, active=1077501952, inactive=11671834624, buffers=2768896, cached=12659343360, shared=901120, slab=959066112)


In [3]:
#导入boto3库
import boto3
#导入sagemaker库
import sagemaker

#创建一个SageMaker Session
sess = sagemaker.Session()
#获取默认S3存储桶
bucket = sess.default_bucket()
#获取当前区域
region = boto3.Session().region_name
#获取账户ID
account_id = boto3.client("sts").get_caller_identity().get("Account")

/opt/conda/lib/python3.7/site-packages/boto3/compat.py:82: PythonDeprecationWarning: Boto3 will no longer support Python 3.7 starting December 13, 2023. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.8 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


In [4]:
from pyathena import connect
import pandas as pd

In [5]:
# 读取已存储为变量 'setup_dependencies_passed'
%store -r setup_dependencies_passed

In [6]:
try:
    setup_dependencies_passed
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN THE PREVIOUS NOTEBOOK ")
    print("You did not install the required libraries.   ")
    print("++++++++++++++++++++++++++++++++++++++++++++++")

In [7]:
print(setup_dependencies_passed)

True


In [8]:
if not setup_dependencies_passed:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN THE PREVIOUS NOTEBOOK ")
    print("You did not install the required libraries.   ")
    print("++++++++++++++++++++++++++++++++++++++++++++++")
else:
    print("[OK]")

[OK]


In [9]:
# 读取已存储为变量 'role'
%store -r role

# 创建Athena表

#### 数据列

- `marketplace`: 2 个字母的国家/地区代码（在本例中全部为 “US”)
- `customer_id`: 随机标识符，可用于汇总单个作者撰写的评论
- `review_id`: 评论的唯一 ID
- `product_id`: 亚马逊标准识别码 (ASIN)  `http://www.amazon.com/dp/<ASIN>` 指向商品详情页面的链接
- `product_parent`: 该 ASIN 的父商品。多个 ASIN（同一商品的颜色或格式变体）可以合并为一个父商品
- `product_title`: 商品的标题描述
- `product_category`: 可用于对评论进行分组的广泛产品类别（在本例中为数字视频）
- `star_rating`: 该评论的评分（1 到 5 星）
- `helpful_votes`: 评论的有用票数
- `total_votes`: 评论获得的总票数
- `vine`: 这篇评论是作为 [Vine](https://www.amazon.com/gp/vine/help) 计划的一部分写的吗？
- `verified_purchase`: 评论来自经过验证的购买吗？
- `review_headline`: 评论的标题
- `review_body`: 评论的文本
- `review_date`: 撰写评论的日期



In [10]:
s3_staging_dir = "s3://{}/athena/query-cache".format(bucket)

In [11]:
database_name = "default"
table_name = "amazon_reviews_parquet"

In [12]:
# 创建Athena连接对象
conn = connect(
    # 指定AWS区域
    region_name=region,
    # 指定S3桶路径用于暂存查询结果
    s3_staging_dir=s3_staging_dir
)

In [15]:
# 定义要执行的 SQL 语句
statement = """
    CREATE EXTERNAL TABLE IF NOT EXISTS {}.{} (
      marketplace string, 
      customer_id string, 
      review_id string, 
      product_id string, 
      product_parent string, 
      product_title string, 
      star_rating int, 
      helpful_votes int, 
      total_votes int, 
      vine string, 
      verified_purchase string, 
      review_headline string, 
      review_body string, 
      review_date int, 
      year int)
    PARTITIONED BY (product_category string)
    ROW FORMAT SERDE 
      'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe' 
    STORED AS INPUTFORMAT 
      'org.apache.hadoop.hive.ql.io.parquet.MapredParquetInputFormat' 
    OUTPUTFORMAT 
      'org.apache.hadoop.hive.ql.io.parquet.MapredParquetOutputFormat'
    LOCATION
      's3://dsoaws/parquet/'
""".format(
    database_name, table_name
)

print(statement)

# 使用pandas的read_sql方法查询Athena
# - statement: 查询的SQL语句
# - conn: 创建的Athena连接对象
# read_sql会发送查询语句到Athena,然后读取返回的结果到pandas DataFrame。通过这种方式可以方便的获取SQL查询结果,并利用pandas进行后续分析处理
pd.read_sql(statement, conn)


    CREATE EXTERNAL TABLE IF NOT EXISTS default.amazon_reviews_parquet (
      marketplace string, 
      customer_id string, 
      review_id string, 
      product_id string, 
      product_parent string, 
      product_title string, 
      star_rating int, 
      helpful_votes int, 
      total_votes int, 
      vine string, 
      verified_purchase string, 
      review_headline string, 
      review_body string, 
      review_date int, 
      year int)
    PARTITIONED BY (product_category string)
    ROW FORMAT SERDE 
      'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe' 
    STORED AS INPUTFORMAT 
      'org.apache.hadoop.hive.ql.io.parquet.MapredParquetInputFormat' 
    OUTPUTFORMAT 
      'org.apache.hadoop.hive.ql.io.parquet.MapredParquetOutputFormat'
    LOCATION
      's3://dsoaws/parquet/'



""


# 分析和构建分区

In [16]:
# 构建MSCK REPAIR TABLE语句,用于刷新Athena表的分区元数据
statement = """
    MSCK REPAIR TABLE {}.{}
""".format(
    database_name, table_name
)

# 打印构建的语句,用于调试
print(statement)

# 执行MSCK REPAIR操作,刷新my_db.my_table的元数据
# 注意read_sql只是执行语句,不会返回实际结果
pd.read_sql(statement, conn)


    MSCK REPAIR TABLE default.amazon_reviews_parquet



""


# 验证表是否成功创建

In [13]:
# 查询数据库中的所有表
statement = "SHOW TABLES in {}".format(database_name)

# 执行语句并获取结果到DataFrame
df_tables = pd.read_sql(statement, conn)

# 打印表名DataFrame
df_tables

,tab_name
0,amazon_reviews_parquet
1,athena_glue_sales
2,cloudtrail_logs_ab2_cloudtrail_logs_9417975856...
3,cloudtrail_logs_location
4,customers_csv
5,customers_parquet
6,r53_rlogs
7,sales_csv
8,sales_parquet


# 运行示例查询

In [14]:
# 定义产品类别变量
product_category = "Digital_Software"

statement = """
    SELECT * FROM {}.{}
    WHERE product_category = '{}' LIMIT 100
""".format(
    database_name, table_name, product_category
)

# 打印生成的SQL语句
print(statement)


    SELECT * FROM default.amazon_reviews_parquet
    WHERE product_category = 'Digital_Software' LIMIT 100



In [15]:
# 使用pandas的read_sql方法,将SQL语句查询结果读入DataFrame
df = pd.read_sql(statement, conn)
# 打印DataFrame的前5行
df.head(5)

,marketplace,customer_id,review_id,product_id,product_parent,product_title,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,year,product_category
0,US,24954530,R2DQXB8NBLNE3W,B00C79J2YM,789114735,Wondershare Video Converter Pro [Download],1,3,4,N,Y,DO NOT BUY!! PIRATED SOFTWARE!!,Bought this for instant download and when I tr...,15899,2013,Digital_Software
1,US,45559297,RHJ1R1KLPUPIZ,B00E7X9RUK,109813655,Quicken Deluxe 20,4,0,0,N,Y,This is a good program and I have used quicken...,This is a good program and I have used quicken...,16391,2014,Digital_Software
2,US,40495668,RIVSJQTLD3FJC,B00CQ7OF6A,102841476,RPG Maker VX Ace,4,0,0,N,N,Excellent Software,"The RPG Maker software is great, especially fo...",15899,2013,Digital_Software
3,US,5010886,R3TQCX31FJSIIX,B0073LIDU8,506184978,Corel AfterShot Pro,1,0,1,N,Y,One Star,the product is crap,16392,2014,Digital_Software
4,US,49312885,R295OIOILJEYH1,B00B1TGHXS,954368001,Microsoft Word 2013 (1PC/1User),1,70,73,N,Y,Not such a good deal...,"Ordered this, thought I would save a little mo...",15901,2013,Digital_Software


In [16]:
if df.empty:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOUR DATA HAS NOT BEEN REGISTERED WITH ATHENA.")
    print("LOOK IN PREVIOUS CELLS TO FIND THE ISSUE.             ")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++")
else:
    print("[OK]")

[OK]


In [17]:
# 判断amazon_reviews_parquet是否在df_tables中的值里,且df不为空。
if "amazon_reviews_parquet" in df_tables.values and not df.empty:
    ingest_create_athena_table_parquet_passed = True

In [18]:
%store ingest_create_athena_table_parquet_passed

Stored 'ingest_create_athena_table_parquet_passed' (bool)


# 查看 Glue 目录中的Athena表

In [23]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="top" href="https://console.aws.amazon.com/glue/home?region={}#table:catalog={};name=amazon_reviews_parquet;namespace=default">AWS Glue Catalog</a></b>'.format(
            region, account_id
        )
    )
)

# 为下一个notebook存储变量

In [19]:
%store

Stored variables and their in-db values:
ingest_create_athena_table_parquet_passed             -> True
model_checkpoint                                      -> 'google/flan-t5-base'
raw_input_data_s3_uri                                 -> 's3://sagemaker-us-east-1-941797585610/data-summar
role                                                  -> 'arn:aws:iam::941797585610:role/service-role/Amazo
s3_private_path_tsv                                   -> 's3://sagemaker-us-east-1-941797585610/amazon-revi
s3_public_path_tsv                                    -> 's3://dsoaws/tsv'
setup_dependencies_passed                             -> True


# Release Resources

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>